In [2]:
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision
from torchvision import transforms


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # kernel=5, paddig=0. stride=1. 32-5+1=28
        self.pool = nn.MaxPool2d(2, 2)  # 14
        self.conv2 = nn.Conv2d(6, 16, 5) # kernel=5, paddig=0. stride=1. 14-5+1=10 => max pooling 후 5X5
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # 84를 60으로 변경 테스트
        self.fc2 = nn.Linear(120, 20)
        self.fc3 = nn.Linear(20, 9)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # 배치를 제외한 모든 차원을 평탄화(flatten)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
net = Net()

In [4]:
import joblib
net = joblib.load('mushroom_test.pkl')

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

trans = transforms.Compose([transforms.Resize((32,32)),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                           ])

imageData = torchvision.datasets.ImageFolder(root = "./Mushrooms_images", transform = trans)

classes = imageData.classes
classes

kfold = KFold(n_splits=5)
for idx, (train_index, test_index) in enumerate(kfold.split(imageData)):
    
    trainset = torch.utils.data.Subset(imageData, train_index)
    testset = torch.utils.data.Subset(imageData, test_index)

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=50, shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(testset, batch_size=50, shuffle=True, num_workers=2)
    
    
    dataiter = iter(trainloader)
    correct = 0
    for i in range(100):
        images, labels = dataiter.next()
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        #print('답: ', labels)
        #print('예측 : ', predicted)
        for j in range(50):  
            if labels[j] == predicted[j]:
                correct += 1

    print(idx, '번째', '트레인 값', correct, '/', (i+1)*50, ',', '%.3f' % (correct/((i+1)*50) * 100), '%') 
    
    dataiter = iter(testloader)
    correct = 0
    for i in range(20):
        images, labels = dataiter.next()
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        #print('답: ', labels)
        #print('예측 : ', predicted)
        for j in range(50):  
            if labels[j] == predicted[j]:
                correct += 1

    print(idx, '번째','테스트 값', correct, '/', (i+1)*50, ',', '%.3f' % (correct/((i+1)*50) * 100), '%')  

0 번째 트레인 값 4787 / 5000 , 95.740 %
0 번째 테스트 값 922 / 1000 , 92.200 %
1 번째 트레인 값 4862 / 5000 , 97.240 %
1 번째 테스트 값 868 / 1000 , 86.800 %
2 번째 트레인 값 4697 / 5000 , 93.940 %
2 번째 테스트 값 993 / 1000 , 99.300 %
3 번째 트레인 값 4709 / 5000 , 94.180 %
3 번째 테스트 값 983 / 1000 , 98.300 %
4 번째 트레인 값 4706 / 5000 , 94.120 %
4 번째 테스트 값 988 / 1000 , 98.800 %


In [6]:
classes = ['Agaricus',
 'Amanita',
 'Boletus',
 'Cortinarius',
 'Entoloma',
 'Hygrocybe',
 'Lactarius',
 'Russula',
 'Suillus']

In [7]:
# state_dict = torch.hub.load_state_dict_from_url
# 링크 받아오는거 
# 파일 업로드 
# 사진 파일을 업로드 -> 테스트 
# https://niceman.tistory.com/150
testimageData = torchvision.datasets.ImageFolder(root = "./test_images", transform = trans)

In [8]:
testImageloader = DataLoader(testimageData,
                        batch_size = 1,
                        shuffle = False,
                        num_workers = 1)

In [9]:
dataiter = iter(testImageloader)

In [10]:
 for i in range(2):
    images, labels = dataiter.next()
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    #print(predicted)
    print(classes[predicted])

Amanita
Russula
